In [1]:
import pynq
from pynq import Overlay
from pynq import MMIO
import time
from operator import*
# Load the overlay (bitstream)
ol = Overlay("spi.bit")
ol?

In [2]:
axi_quad_spi=ol.axi_quad_spi_0
axi_quad_spi?

In [ ]:
axi_quad_spi.register_map.SPICR.SPE =1

In [45]:
# Access the IP dictionary to find the address of the axi_quad_spi_0
spi_ip = ol.ip_dict['axi_quad_spi_0']
spi_base_address = (spi_ip['phys_addr'])
spi_address_range = (spi_ip['addr_range']) 

print(spi_base_address)

1105199104


In [46]:
#spi_base_address = 0x41E00000
#spi_address_range= 0x1000
# Set up MMIO for AXI Quad SPI
spi_mmio = MMIO(spi_address_range, spi_address_range)
spi_mmio.debug = True

In [47]:

# Define the register offsets relative to the base address
SPI_SPICR_OFFSET = 0x60  # SPI Control Register
SPI_SPISR_OFFSET = 0x64  # SPI Status Register
SPI_TXD_OFFSET   = 0x68  # SPI Transmit FIFO
SPI_RXD_OFFSET   = 0x78  # SPI Receive FIFO
SPI_SPISSR_OFFSET = 0x70  # SPI Slave Select Register
# Configure SPI Control Register (SPICR) with a value (enable SPI, master mode, etc.)
SPICR_ENABLE = 0x00000086  # Enable SPI in master mode, reset FIFOs, etc.




In [48]:
spi_mmio.write(SPI_SPICR_OFFSET, SPICR_ENABLE)

MMIO Debug: Writing 4 bytes to offset 60: 86


In [49]:
# Function to perform SPI read operation
def spi_read(address, num_bytes):

    # Set the SPI control register (example settings from previous response)
    spi_mmio.write(SPI_SPICR_OFFSET, 0x0183)  # Master mode, SPI enabled, manual SS, etc.

    # Assert the correct SPI slave (modify based on how your hardware is configured)
    spi_mmio.write(SPI_SPISSR_OFFSET, 0xFFFFFFFE)  # Assert the first slave (bit 0 low)

    # Write the address to the TX FIFO
    spi_mmio.write(SPI_TXD_OFFSET, address)
 

    # Read the data from the RX FIFO
    read_data = []
    for _ in range(num_bytes):
        spi_mmio.write(SPI_TXD_OFFSET, 0x0000) 
        read_data.append(spi_mmio.read(SPI_RXD_OFFSET))

    # Deassert the slave (set all bits high)
    spi_mmio.write(SPI_SPISSR_OFFSET, 0xFFFFFFFF)

    return read_data




In [50]:
# Example usage
address = 0x00  # Replace with the actual address/command to read from
num_bytes = 20   # Number of bytes to read
data = spi_read(address, num_bytes)

print("Data read from SPI device:", data)

MMIO Debug: Writing 4 bytes to offset 60: 183
MMIO Debug: Writing 4 bytes to offset 70: fffffffe
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from offset 78
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from offset 78
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from offset 78
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from offset 78
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from offset 78
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from offset 78
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from offset 78
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from offset 78
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from offset 78
MMIO Debug: Writing 4 bytes to offset 68: 0
MMIO Debug: Reading 4 bytes from

In [51]:
# Read the SPI Status Register (SPISR)
status = spi_mmio.read(SPI_SPISR_OFFSET)
print(f"SPI Status Register: {hex(status)}")

MMIO Debug: Reading 4 bytes from offset 64
SPI Status Register: 0x0
